In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# standard imports
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers
import random
from sklearn import preprocessing
from keras.callbacks import History
import matplotlib.pyplot as plt
from keras.utils import np_utils
import os
from tensorflow.keras.optimizers import Adam
from plain_neural_network import*
from keras import backend as K

%matplotlib inline

2023-10-11 20:10:26.226187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_dir = "/albedo/home/ssunar/CNN_eddy_detection/for_paper/interpolation/south_atlantic/months"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])
#input_file_paths.pop(29)
#input_file_paths.pop(11)

data_x = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_x = data_x.ssh.to_numpy()
X = np.float32(data_x)
                            
X[X>1000] = 0
print(X.shape)

(365, 840, 480)


In [3]:
input_dir = "/albedo/home/ssunar/CNN_eddy_detection/for_paper/segmentation_masks/south_atlantic"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])
#input_file_paths.pop(29)
#input_file_paths.pop(11)

data_y = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_y = data_y.seg_mask.to_numpy()
Y = np.float32(data_y)


(Y.shape)

(365, 840, 480)

In [4]:
Y[(Y != 1) & (Y!=2)] = 0

In [15]:
def split_3d_vector(vector):
    X, Y, Z = vector.shape
    chunks = []
    
    for i in range(X):
        vector_2d = vector[i,:,:]
        B = Y // 256 + 1
        for b in range(B):
            start_y = b * 256
            end_y = (b + 1) * 256
            if end_y > Y:
                start_y = Y - 256
                end_y = Y
      
            C = Z // 256 + 1
            for c in range(C):
                start_z = c * 256
                end_z = (c + 1) * 256
                if end_z > Z:
                    start_z = Z - 256
                    end_z = Z
                final_chunk = vector_2d[start_y:end_y, start_z:end_z]
                chunks.append(final_chunk)
    return chunks

In [18]:
new_X = np.array(split_3d_vector(X))
(new_X.shape)

new_Y = np.array(split_3d_vector(Y))
(new_Y.shape)

(2920, 256, 256)

(2920, 256, 256)

In [6]:
"""
data_x_1 = data_x
data_y_1 = data_y

#only using a portion of data from this region
used_split = 0.4
used_samples = int(used_split*len(data_x))
#same seed must be used
random.Random(0).shuffle(data_x)
random.Random(0).shuffle(data_y)
data_x_1 = data_x[0:used_samples]
data_y_1 = data_y[0:used_samples]
"""

'\ndata_x_1 = data_x\ndata_y_1 = data_y\n\n#only using a portion of data from this region\nused_split = 0.4\nused_samples = int(used_split*len(data_x))\n#same seed must be used\nrandom.Random(0).shuffle(data_x)\nrandom.Random(0).shuffle(data_y)\ndata_x_1 = data_x[0:used_samples]\ndata_y_1 = data_y[0:used_samples]\n'

In [7]:
"""
input_dir = "/work/ollie/bpanthi/NN_gulf/"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])
input_file_paths.pop(6)
data_x_2 = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_x_2 = data_x_2.ssh.to_numpy()
data_x_2 = np.float32(data_x_2)
                            
data_x_2[data_x_2>1000] = 0
data_x_2 = data_x_2[:,52:308, 52:308]
print(data_x_2.shape)

input_dir = "/home/ollie/ssunar/segmentation_masks_NN_gulf"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])
input_file_paths.pop(6)
data_y_2 = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_y_2 = data_y_2.seg_mask.to_numpy()
data_y_2 = np.float32(data_y_2)
data_y_2 = data_y_2[:,52:308, 52:308]
print(data_y_2.shape)
"""

'\ninput_dir = "/work/ollie/bpanthi/NN_gulf/"\ninput_file_paths = sorted(\n    [\n        os.path.join(input_dir, fname)\n        for fname in os.listdir(input_dir)\n    ])\ninput_file_paths.pop(6)\ndata_x_2 = xr.open_mfdataset(input_file_paths,combine = \'nested\', concat_dim="TIME")\ndata_x_2 = data_x_2.ssh.to_numpy()\ndata_x_2 = np.float32(data_x_2)\n                            \ndata_x_2[data_x_2>1000] = 0\ndata_x_2 = data_x_2[:,52:308, 52:308]\nprint(data_x_2.shape)\n\ninput_dir = "/home/ollie/ssunar/segmentation_masks_NN_gulf"\ninput_file_paths = sorted(\n    [\n        os.path.join(input_dir, fname)\n        for fname in os.listdir(input_dir)\n    ])\ninput_file_paths.pop(6)\ndata_y_2 = xr.open_mfdataset(input_file_paths,combine = \'nested\', concat_dim="TIME")\ndata_y_2 = data_y_2.seg_mask.to_numpy()\ndata_y_2 = np.float32(data_y_2)\ndata_y_2 = data_y_2[:,52:308, 52:308]\nprint(data_y_2.shape)\n'

In [8]:
"""
input_dir = "/work/ollie/bpanthi/NN_kursaal/"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])

data_x_3 = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_x_3 = data_x_3.ssh.to_numpy()
data_x_3 = np.float32(data_x_3)
                            
data_x_3[data_x_3>1000] = 0
data_x_3 = data_x_3[:,52:308, 52:308]
print(data_x_3.shape)

input_dir = "/home/ollie/ssunar/segmentation_masks_NN_kursaal"
input_file_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ])
data_y_3 = xr.open_mfdataset(input_file_paths,combine = 'nested', concat_dim="TIME")
data_y_3 = data_y_3.seg_mask.to_numpy()
data_y_3 = np.float32(data_y_3)
data_y_3 = data_y_3[:,52:308, 52:308]
print(data_y_3.shape)
"""

'\ninput_dir = "/work/ollie/bpanthi/NN_kursaal/"\ninput_file_paths = sorted(\n    [\n        os.path.join(input_dir, fname)\n        for fname in os.listdir(input_dir)\n    ])\n\ndata_x_3 = xr.open_mfdataset(input_file_paths,combine = \'nested\', concat_dim="TIME")\ndata_x_3 = data_x_3.ssh.to_numpy()\ndata_x_3 = np.float32(data_x_3)\n                            \ndata_x_3[data_x_3>1000] = 0\ndata_x_3 = data_x_3[:,52:308, 52:308]\nprint(data_x_3.shape)\n\ninput_dir = "/home/ollie/ssunar/segmentation_masks_NN_kursaal"\ninput_file_paths = sorted(\n    [\n        os.path.join(input_dir, fname)\n        for fname in os.listdir(input_dir)\n    ])\ndata_y_3 = xr.open_mfdataset(input_file_paths,combine = \'nested\', concat_dim="TIME")\ndata_y_3 = data_y_3.seg_mask.to_numpy()\ndata_y_3 = np.float32(data_y_3)\ndata_y_3 = data_y_3[:,52:308, 52:308]\nprint(data_y_3.shape)\n'

In [9]:
"""
data_x = np.concatenate((data_x_1, np.concatenate((data_x_2,data_x_3), axis=0)), axis=0)
data_y = np.concatenate((data_y_1, np.concatenate((data_y_2,data_y_3), axis=0)), axis=0)
print(data_x.shape)
print(data_y.shape)
"""

'\ndata_x = np.concatenate((data_x_1, np.concatenate((data_x_2,data_x_3), axis=0)), axis=0)\ndata_y = np.concatenate((data_y_1, np.concatenate((data_y_2,data_y_3), axis=0)), axis=0)\nprint(data_x.shape)\nprint(data_y.shape)\n'

In [19]:
img_size = (256, 256)
num_classes = 3
batch_size = 16
epochs = 90
total_samples = len(data_x)
print(total_samples)

2190


In [20]:
model = get_model(img_size, num_classes)
model.summary()

2023-10-11 20:51:03.812220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 separable_conv2d (SeparableCon  (None, 256, 256, 16  25         ['input_1[0][0]']                
 v2D)                           )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['separable_conv2d[0][0]']       
 alization)                     )                                                             

                                                                                                  
 batch_normalization_7 (BatchNo  (None, 32, 32, 32)  128         ['separable_conv2d_7[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 activation_7 (Activation)      (None, 32, 32, 32)   0           ['batch_normalization_7[0][0]']  
                                                                                                  
 dropout_3 (Dropout)            (None, 32, 32, 32)   0           ['activation_7[0][0]']           
                                                                                                  
 up_sampling2d (UpSampling2D)   (None, 64, 64, 32)   0           ['dropout_3[0][0]']              
                                                                                                  
 concatena

                                )                                                                 
                                                                                                  
 separable_conv2d_14 (Separable  (None, 256, 256, 3)  64         ['dropout_6[0][0]']              
 Conv2D)                                                                                          
                                                                                                  
 reshape (Reshape)              (None, 65536, 3)     0           ['separable_conv2d_14[0][0]']    
                                                                                                  
 activation_14 (Activation)     (None, 65536, 3)     0           ['reshape[0][0]']                
                                                                                                  
Total params: 13,897
Trainable params: 13,257
Non-trainable params: 640
_____________________________________

In [21]:
#Loss function 
#defined from the paper:
#Santana et al._2020_Neural network training for the detection and classification of oceanic mesoscale eddies

unique, counts = np.unique(data_y, return_counts=True)
dict(zip(unique, counts))

freq = [np.sum(counts)/j for j in counts]
weightsSeg = [f/np.sum(freq) for f in freq]
print(weightsSeg)

def dice_coef_anti(y_true, y_pred):
    smooth = 1.  # to avoid zero division
    y_true_anti = y_true[:,:,1]
    y_pred_anti = y_pred[:,:,1]
    intersection_anti = K.sum(y_true_anti * y_pred_anti)
    return (2 * intersection_anti + smooth) / (K.sum(y_true_anti)+ K.sum(y_pred_anti) + smooth)

def dice_coef_cyc(y_true, y_pred):
    smooth = 1.  # to avoid zero division
    y_true_cyc = y_true[:,:,2]
    y_pred_cyc = y_pred[:,:,2]
    intersection_cyc = K.sum(y_true_cyc * y_pred_cyc)
    return (2 * intersection_cyc + smooth) / (K.sum(y_true_cyc) + K.sum(y_pred_cyc) + smooth)

def dice_coef_nn(y_true, y_pred):
    smooth = 1.  # to avoid zero division
    y_true_nn = y_true[:,:,0]
    y_pred_nn = y_pred[:,:,0]
    intersection_nn = K.sum(y_true_nn * y_pred_nn)
    return (2 * intersection_nn + smooth) / (K.sum(y_true_nn) + K.sum(y_pred_nn) + smooth)
    
def mean_dice_coef(y_true, y_pred):
    return (dice_coef_anti(y_true, y_pred) + dice_coef_cyc(y_true, y_pred) + dice_coef_nn(y_true, y_pred))/3.

def weighted_mean_dice_coef(y_true, y_pred):
    #return (weightsSeg[2]*dice_coef_anti(y_true, y_pred) + weightsSeg[1]*dice_coef_cyc(y_true, y_pred) + weightsSeg[0]*dice_coef_nn(y_true, y_pred))
    return (weightsSeg[2]*dice_coef_anti(y_true, y_pred) + weightsSeg[1]*dice_coef_cyc(y_true, y_pred) + weightsSeg[0]*dice_coef_nn(y_true, y_pred))
      
def dice_coef_loss(y_true, y_pred):
    return 1 - weighted_mean_dice_coef(y_true, y_pred)

{0.0: 127868244, 1.0: 7466151, 2.0: 8189445}

[0.029638252979173923, 0.5075963992256167, 0.4627653477952094]


In [22]:
# Split our img paths into a training and a validation set
split = 0.2
train_samples = int((1-split)*total_samples)
#same seed must be used
random.Random(0).shuffle(data_x)
random.Random(0).shuffle(data_y)
train_input = data_x[0:train_samples]
train_target = data_y[0:train_samples]
val_input = data_x[train_samples:total_samples]
val_target = data_y[train_samples:total_samples]

print("train_input:", train_input.shape)
print("val_input:", val_input.shape)

# Instantiate data Sequences for each split
train_gen = plain_net_eddy(batch_size, img_size, train_input, train_target)
val_gen = plain_net_eddy(batch_size, img_size, val_input, val_target)
print("Size of each batch: ",train_gen[1][0].shape)

train_input: (1752, 256, 256)
val_input: (438, 256, 256)
Size of each batch:  (16, 256, 256, 1)


In [ ]:
file_path_save = "/albedo/home/ssunar/CNN_eddy_detection/for_paper/unet_trained/south_atlantic"
model.compile(optimizer=Adam(lr=1e-3), loss=dice_coef_loss, metrics=['categorical_accuracy', mean_dice_coef, weighted_mean_dice_coef])

callbacks = [keras.callbacks.ModelCheckpoint(file_path_save, save_best_only=True , monitor='val_loss',save_weights_only=True, save_freq="epoch"),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto', min_delta=1e-30, min_lr=1e-30)]

# Train the model, doing validation at the end of each epoch.
history = model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks, shuffle=True,verbose=1)

Epoch 1/90
109/109 [==============================] - 430s 4s/step - loss: 0.8448 - categorical_accuracy: 0.6110 - mean_dice_coef: 0.2984 - weighted_mean_dice_coef: 0.1552 - val_loss: 0.8931 - val_categorical_accuracy: 0.8885 - val_mean_dice_coef: 0.2852 - val_weighted_mean_dice_coef: 0.1069 - lr: 0.0010
Epoch 2/90
109/109 [==============================] - 426s 4s/step - loss: 0.6986 - categorical_accuracy: 0.7342 - mean_dice_coef: 0.4557 - weighted_mean_dice_coef: 0.3014 - val_loss: 0.9385 - val_categorical_accuracy: 0.8885 - val_mean_dice_coef: 0.3293 - val_weighted_mean_dice_coef: 0.0615 - lr: 0.0010
Epoch 3/90
109/109 [==============================] - 425s 4s/step - loss: 0.5515 - categorical_accuracy: 0.8709 - mean_dice_coef: 0.5947 - weighted_mean_dice_coef: 0.4485 - val_loss: 0.9690 - val_categorical_accuracy: 0.8885 - val_mean_dice_coef: 0.3153 - val_weighted_mean_dice_coef: 0.0310 - lr: 0.0010
Epoch 4/90
109/109 [==============================] - 425s 4s/step - loss: 0.4775 

Epoch 28/90
109/109 [==============================] - 424s 4s/step - loss: 0.3183 - categorical_accuracy: 0.9359 - mean_dice_coef: 0.7706 - weighted_mean_dice_coef: 0.6817 - val_loss: 0.3896 - val_categorical_accuracy: 0.9173 - val_mean_dice_coef: 0.7185 - val_weighted_mean_dice_coef: 0.6104 - lr: 0.0010
Epoch 29/90
109/109 [==============================] - 424s 4s/step - loss: 0.3156 - categorical_accuracy: 0.9364 - mean_dice_coef: 0.7725 - weighted_mean_dice_coef: 0.6844 - val_loss: 0.4278 - val_categorical_accuracy: 0.9192 - val_mean_dice_coef: 0.6938 - val_weighted_mean_dice_coef: 0.5722 - lr: 0.0010
Epoch 30/90
109/109 [==============================] - 424s 4s/step - loss: 0.3175 - categorical_accuracy: 0.9361 - mean_dice_coef: 0.7712 - weighted_mean_dice_coef: 0.6825 - val_loss: 0.4878 - val_categorical_accuracy: 0.9072 - val_mean_dice_coef: 0.6499 - val_weighted_mean_dice_coef: 0.5122 - lr: 0.0010
Epoch 31/90
109/109 [==============================] - 424s 4s/step - loss: 0.3

Epoch 53/90
109/109 [==============================] - 424s 4s/step - loss: 0.2900 - categorical_accuracy: 0.9417 - mean_dice_coef: 0.7910 - weighted_mean_dice_coef: 0.7100 - val_loss: 0.4029 - val_categorical_accuracy: 0.9209 - val_mean_dice_coef: 0.7111 - val_weighted_mean_dice_coef: 0.5971 - lr: 2.5000e-04
Epoch 54/90
109/109 [==============================] - 424s 4s/step - loss: 0.2896 - categorical_accuracy: 0.9418 - mean_dice_coef: 0.7913 - weighted_mean_dice_coef: 0.7104 - val_loss: 0.3940 - val_categorical_accuracy: 0.9242 - val_mean_dice_coef: 0.7179 - val_weighted_mean_dice_coef: 0.6060 - lr: 2.5000e-04
Epoch 55/90
109/109 [==============================] - 425s 4s/step - loss: 0.2893 - categorical_accuracy: 0.9418 - mean_dice_coef: 0.7915 - weighted_mean_dice_coef: 0.7107 - val_loss: 0.4223 - val_categorical_accuracy: 0.9219 - val_mean_dice_coef: 0.6977 - val_weighted_mean_dice_coef: 0.5777 - lr: 2.5000e-04
Epoch 56/90
109/109 [==============================] - 424s 4s/step

In [ ]:
dir_path = "/albedo/home/ssunar/CNN_eddy_detection/for_paper/unet_trained/south_atlantic"
plt.figure(figsize=(10, 10))
plt.semilogy(model.history.history['loss'])
plt.semilogy(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'], loc='upper right');
plt.savefig(dir_path + "loss_graph.png", dpi = 300)

In [ ]:
import time
from sklearn.metrics import classification_report
start = time.time()
val_gen = plain_net_eddy(batch_size, img_size, val_input, val_target)
val_preds = model.predict(val_gen)
print(val_preds.shape)

end = time.time()
time_per_img = (end - start)/438
print("Time Per image:",time_per_img)
print("Time for 1 year eqv:",time_per_img*365)

In [ ]:
# val_gen = plain_net_eddy(batch_size, img_size, val_input, val_target)
# mask = np.expand_dims(val_preds, axis=-1)
val_gen = data_x[10]
val_gen = np.reshape(val_gen,(1,img_size[0],img_size[1],1))
print(val_gen.shape)
val_preds = model.predict(val_gen)
print(val_preds.shape)
mask = np.argmax(np.reshape(val_preds[0],(1,img_size[1],img_size[0],3)), axis=-1)
mask = np.reshape(mask,(img_size[0],img_size[1]))
print(classification_report(np.reshape(data_y[10],(65536)),np.reshape(mask,(65536))))

In [ ]:
#seeing the prediction of one image of the validation set
%matplotlib inline

plt.figure(figsize=(20, 10))

plt.subplot(131)
plt.imshow(mask.T, cmap='viridis')
plt.colorbar(extend='both', fraction=0.042, pad=0.04)
plt.axis('off')
plt.title('Result Segmentation');
plt.savefig(dir_path + "prediction.png", dpi = 300)

plt.subplot(132)
plt.imshow(data_y[10].T, cmap='viridis')
plt.colorbar(extend='both', fraction=0.042, pad=0.04)
plt.axis('off')
plt.title('Groundtruth Segmentation');
plt.savefig(dir_path + "groundtruth.png", dpi = 300)